# Lesson 2.2:
# PowerGrid Models API - Using JSON Queries

This tutorial introduces the PowerGrid Models API and how it can be used to query model data.

__Learning Objectives:__

At the end of the tutorial, the user should be able to use the PowerGrid Models API to

* 
* 
* 

## Getting Started

Before running any of the sample routines in this tutorial, it is first necessary to start the GridAPPS-D Platform and establish a connection to this notebook so that we can start passing calls to the API.

_Open the Ubuntu terminal and start the GridAPPS-D Platform if it is not running already:_

`cd gridappsd-docker`

~/gridappsd-docker$ `./run.sh -t develop`

_Once containers are running,_

gridappsd@[container]:/gridappsd$ `./run-gridappsd.sh`

In [1]:
# Establish connection to GridAPPS-D Platform:
from gridappsd import GridAPPSD
gapps = GridAPPSD("('localhost', 61613)", username='system', password='manager')
model_mrid = "_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62" # IEEE 13 Node used for all example queries

---
# Table of Contents

* [1. Introduction to the PowerGrid Model API](#1.-Introduction-to-the-PowerGrid-Model-API)
* [2. Using the PowerGrid Model API](#2.-Using-the-PowerGrid-Model-API)
    * [2.1. Specifying the Topic](#2.1.-Specifying-the-Topic)
    * [2.2. Structure of a JSON Query Message](#2.2.-Structure-of-a-JSON-Query-Message)
    * [2.3. Specifying the requestType](#2.3.-Specifying-the-requestType)
* [3. Querying for Feeder Model Info](#3.-Querying-for-Feeder-Model-Info)
    * [3.1. Query for mRIDs of all Models](#3.1.-Query-for-mRIDs-of-all-Models)
    * [3.2. Query for Details Dictionary of all Models](#3.2.-Query-for-Details-Dictionary-of-all-Models)
* [4. Querying for Object Info](#4.-Querying-for-Object-Info)
    * [4.1. Query for CIM Classes of Objects in Model](#4.1.-Query-for-CIM-Classes-of-Objects-in-Model)

---
# 1. Introduction to the PowerGrid Model API


The PowerGrid Model API is used to pull model information, inlcuding the names, mRIDs, measurements, and nominal values of power system equipment in the feeder (such as lines, loads, switches, transformers, and DERs).

INSERT MORE ON API

INSERT MORE ON API

INSERT MORE ON API

---
# 2. Using the PowerGrid Model API

## 2.1. Specifying the Topic

All queries passed to the PowerGrid Models API need to use the correct topic.For a review of GridAPPS-D topics, see Lesson 1.4 There are two ways to specify the topic. Both produce identical results.

__1) Specifying the topic as a string:__

In [2]:
topic = "goss.gridappsd.process.request.data.powergridmodel"

__2) Using the `topics` library to specify the topic:__

In [ ]:
from gridappsd import topics as t
topic = t.REQUEST_POWERGRID_DATA

[Return to Top](#Table-of-Contents)

## 2.2. Structure of a JSON Query Message

Most simple queries are passed to PowerGrid Models API as JSON scripts wrapped in python string. The general format is 

```
message = """
{
    "requestType": "INSERT QUERY HERE",
    "resultFormat": "JSON",
    "modelId": "OPTIONAL INSERT MODEL mRID HERE",
    "objectId": "OPTIONAL INSERT OBJECT mRID HERE",
    "filter": "OPTIONAL INSERT SPARQL FILTER HERE"
}
```

The components of the message are as follows:

* `"requestType":` -- Specifies the type of query. Available requestType are listed in the next section.


* `"resultFormat":` -- Specifies the format of the response, can be `"JSON"`, `"CSV"`, or `"XML"`. (CAUTION: the PowerGridModel API uses the key _resultFormat_, while the Timeseries API uses the key _reponseFormat_. Using the wrong key for either API will result in a java.lang error.)


* `"modelID":` -- Optional. Used to filter the query to only one particular model whose mRID is specified. Be aware of spelling and capitalization differences between JSON query spelling `"modelId"` and Python Library spelling `model_id`.


* `"objectType":` -- Optional. Used to filter the query to only one CIM class of equipment. Speciying the _objectID_ will override any values specified for _objectType_. 


* `"objectID":` -- Optional. Used to filter the query to only one object whose mRID is specified. Specifying the _objectID_ will override any values specified for _objectType_. 


* `"filter":` -- Optional. Used to filter the query using a SPARQL filter. SPARQL queries are covered in the next lesson.


The usage of each of these message components are explained in detail with code block examples below. 

__Important__: Be sure to pay attention to placement of commas ( __,__ ) at the end of each JSON line. Commas are placed at the end of each line _except_ the last line. Incorrect comma placement will result in a JsonSyntaxException. 

All of the queries are passed to the PowerGrid Model API using the `.get_response(topic, message)` method for the GridAPPS-D platform connection variable.

[Return to Top](#Table-of-Contents)

## 2.3. Specifying the `requestType`

Below are the possible `requestType` strings that are used to specify the type of each query. Executable code block examples are provided for each of the requests in the subsections below.

The first group of _requestType_ key-value pairs are for queries for information related to the entire model or a set of models, such as the model name, mRID, region, and substation:

* `"requestType": "QUERY_MODEL_NAMES"` -- [Query for the list of all model name mRIDs](#3.1.-Query-for-mRIDs-of-all-Models)


* `"requestType": "QUERY_MODEL_INFO"` -- [Query for the dictionary of all details for all feeders in Blazegraph](#3.2.-Query-for-Details-Dictionary-of-all-Models)


The second group of _requestType_ key-value pairs are for queries for a single object or a single class of objects withing a model, such as the object mRID, CIM attributes, or measurement points:

* `"requestType": "QUERY_OBJECT_TYPES"` -- [Query for the types of CIM classes of objects in the model](#4.1.-Query-for-CIM-Classes-of-Objects-in-Model)


* `"requestType": "QUERY_OBJECT_IDS"` -- Query for a list of all mRIDs for objects of a CIM class in the model


* `"requestType": "QUERY_OBJECT"` -- Query for CIM attributes of an object using its unique mRID 


* `"requestType": "QUERY_OBJECT_DICT"` -- Query for the dictionary of all details for an object using either its _objectType_ OR its _objectID_


* `"requestType": "QUERY_OBJECT_MEASUREMENTS"` -- Query for all measurement types and mRIDs for an object using either its _objectType_ OR its _ObjectID_.

The third group of _requestType_ key-value pairs are for queries based on SPARQL filters or complete SPARQL queries. The structure of SPARQL was introduced in [Lesson 1.XX](). Usage of these two _requestType_ will covered separately in the next two lessons.

* `"requestType": "QUERY_MODEL"` -- Query for all  part of a specified model, filtered by object type using a SPARQL filter.


* `"requestType": "QUERY"` -- Query using a complete SPARQL query.




[[Return to Top](#Table-of-Contents)]

---
# 3. Querying for Feeder Model Info


This section outlines the pre-built JSON queries that can be passed to the PowerGrid Model API to obtain mRIDs and other information for all models and feeders stored in the Blazegraph Database.

## 3.1. Query for mRIDs of all Models

This query obtains a list of all the model MRIDs stored in the Blazegraph database. 

Query requestType:

* `"requestType": "QUERY_MODEL_NAMES"`

Allowed parameters:

* `"resultFormat":` – "XML" / "JSON" / "CSV" -- Optional. Will return results as a list in the format selected. 

In [7]:
message = """{
    "requestType": "QUERY_MODEL_NAMES",
    "resultFormat": "JSON"
}"""

In [4]:
gapps.get_response(topic, message)

{'data': {'modelNames': ['_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62',
   '_4F76A5F9-271D-9EB8-5E31-AA362D86F2C3',
   '_503D6E20-F499-4CC7-8051-971E23D0BF79',
   '_5B816B93-7A5F-B64C-8460-47C17D6E4B0F',
   '_67AB291F-DCCD-31B7-B499-338206B9828F',
   '_77966920-E1EC-EE8A-23EE-4EFD23B205BD',
   '_9CE150A8-8CC5-A0F9-B67E-BBD8C79D3095',
   '_AAE94E4A-2465-6F5E-37B1-3E72183A4E44',
   '_C1C3E687-6FFD-C753-582B-632A27E28507',
   '_E407CBB6-8C8D-9BC9-589C-AB83FBF0826D']},
 'responseComplete': True,
 'id': '736502389'}

In [8]:
type(message)

str

[Return to Top](#Table-of-Contents)

## 3.2. Query for Details Dictionary of all Models

This query returns a list of names and MRIDs for all models, substations, subregions, and regions for all available feeders stored in the Blazegraph database.

Query requestType:

* `"requestType": "QUERY_MODEL_INFO"`

Allowed parameters:

* `"resultFormat":` – "XML" / "JSON" / "CSV" -- Will return results as a list in the format selected.

In [ ]:
message = """
{
    "requestType": "QUERY_MODEL_INFO",
    "resultFormat": "JSON"
}
"""

In [ ]:
gapps.get_response(topic, message)

[Return to Top](#Table-of-Contents)

---
# 4. Querying for Object Info

This section outlines the pre-built JSON queries that can be passed to the PowerGrid Model API to obtain mRIDs and other information for a particular object or a class of objects for one or more feeders stored in the Blazegraph Database.

All of the examples in this section use the IEEE 13 node model. The python constructor %s is used for all queries to enable the code block to be cut and paste into any python script without needing to change the model mRID.

In [10]:
model_mrid = "_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62" # IEEE 13 Node used for all example queries

## 4.1. Query for CIM Classes of Objects in Model

This query is used to query for a list of all the CIM XML classes of objects present in the Blazegraph for a particular model or all models in the database.

Query requestType is

* `"requestType": "QUERY_OBJECT_TYPES"`

Allowed parameters are

* `"modelId":` "model name mRID" -- Optional. Searches only the particular model identified by the given unique mRID
* `"resultFormat":` – "XML" / "JSON" / "CSV" -- Will return results as a list in the format selected. 

__1) Query entire Blazegraph database__

Omit the "modelId" parameter to search the entire blazegraph database. 

In [12]:
message = """
{
    "requestType": "QUERY_OBJECT_TYPES",
    "resultFormat": "JSON"
}
"""

In [13]:
gapps.get_response(topic, message)

{'data': {'objectTypes': ['http://iec.ch/TC57/CIM100#ACLineSegment',
   'http://iec.ch/TC57/CIM100#BaseVoltage',
   'http://iec.ch/TC57/CIM100#BatteryUnit',
   'http://iec.ch/TC57/CIM100#ConnectivityNode',
   'http://iec.ch/TC57/CIM100#CoordinateSystem',
   'http://iec.ch/TC57/CIM100#CurrentLimit',
   'http://iec.ch/TC57/CIM100#EnergyConsumer',
   'http://iec.ch/TC57/CIM100#EnergySource',
   'http://iec.ch/TC57/CIM100#Feeder',
   'http://iec.ch/TC57/CIM100#GeographicalRegion',
   'http://iec.ch/TC57/CIM100#IEC61970CIMVersion',
   'http://iec.ch/TC57/CIM100#LoadResponseCharacteristic',
   'http://iec.ch/TC57/CIM100#Location',
   'http://iec.ch/TC57/CIM100#OperationalLimitSet',
   'http://iec.ch/TC57/CIM100#OperationalLimitType',
   'http://iec.ch/TC57/CIM100#PerLengthSequenceImpedance',
   'http://iec.ch/TC57/CIM100#PhotovoltaicUnit',
   'http://iec.ch/TC57/CIM100#PositionPoint',
   'http://iec.ch/TC57/CIM100#PowerElectronicsConnection',
   'http://iec.ch/TC57/CIM100#PowerTransformer',


__2) Query for only a particular model__

Specify the model MRID as a python string and pass it as a parameter to the method to return only the CIM classes of objects in that particular model. 

Be aware of spelling and capitalization differences between JSON query spelling `"modelId"` and Python Library spelling `model_id`.

In [11]:
message = """
{
    "requestType": "QUERY_OBJECT_TYPES",
    "modelId": "%s",
    "resultFormat": "JSON"
}
""" % model_mrid

In [12]:
gapps.get_response(topic, message)

{'data': {'objectTypes': ['http://iec.ch/TC57/CIM100#ConnectivityNode',
   'http://iec.ch/TC57/CIM100#ACLineSegment',
   'http://iec.ch/TC57/CIM100#EnergyConsumer',
   'http://iec.ch/TC57/CIM100#TransformerTank',
   'http://iec.ch/TC57/CIM100#PowerTransformer',
   'http://iec.ch/TC57/CIM100#LoadBreakSwitch',
   'http://iec.ch/TC57/CIM100#Fuse',
   'http://iec.ch/TC57/CIM100#Breaker',
   'http://iec.ch/TC57/CIM100#PowerElectronicsConnection',
   'http://iec.ch/TC57/CIM100#EnergySource',
   'http://iec.ch/TC57/CIM100#LinearShuntCompensator',
   'http://iec.ch/TC57/CIM100#Recloser']},
 'responseComplete': True,
 'id': '380248986'}

[Return to Top](#Table-of-Contents)

## 4.2. Query for mRIDs of Objects in a Feeder

This query is used to obtain all the mRIDs of objects of a particular CIM class in the feeder. 

Query responseType is

* `"requestType": "QUERY_OBJECT_IDS"`

Allowed parameters are:

* `"modelId":` "model name mRID" -- When specified it searches against that model, if empty it will search against all models
* `"objectType":` "CIM Class" -- Optional. Specifies the type of objects you wish to return details for.
* `"resultFormat":` – "XML" / "JSON" / "CSV" -- Will return results as a list in the format selected. 

Within a particular feeder, it is possible to query for objects of all the CIM classes obtained using `"requestType": "QUERY_OBJECT_TYPES"` (discussed above in [Section 4.1](#4.1.-Query-for-CIM-Classes-of-Objects-in-Model)). Note that the RDF URI is not included in the query, only the name of the class, such as `"objectType": "ACLineSegment"` or `"objectType": "LoadBreakSwitch"`.

In [19]:
message = {
    "requestType": "QUERY_OBJECT_IDS",
    "resultFormat": "JSON",
    "modelId": model_mrid,
    "objectType": "LoadBreakSwitch"
} 

In [20]:
gapps.get_response(topic, message)

{'data': {'objectIds': ['_2858B6C2-0886-4269-884C-06FA8B887319',
   '_517413CB-6977-46FA-8911-C82332E42884']},
 'responseComplete': True,
 'id': '33519682'}

[Return to Top](#Table-of-Contents)

## 4.3. Query for CIM Attributes of an Object

This query is used to obtain all the attributes and mRIDs of those attributes for a particular object whose mRID is specified. 

Query responseType is

* `"requestType": "QUERY_OBJECT"`

Allowed parameters are:

* `"modelId":` "model name mRID" -- When specified it searches against that model, if empty it will search against all models
* `"objectId":` "object mRID" -- Optional. Specifies the type of objects you wish to return details for.
* `"resultFormat":` – "XML" / "JSON" / "CSV" -- Will return results as a list in the format selected.

Within a particular feeder, it is possible to query for objects of all the CIM classes obtained using `"requestType": "QUERY_OBJECT_TYPES"` (discussed above in [Section 4.1](#4.1.-Query-for-CIM-Classes-of-Objects-in-Model)). Note that the RDF URI is not included in the query, only the name of the class, such as `"objectType": "ACLineSegment"` or `"objectType": "LoadBreakSwitch"`.

In [28]:
object_mrid = "_2858B6C2-0886-4269-884C-06FA8B887319"
message = """
{
        "requestType": "QUERY_OBJECT",
        "resultFormat": "JSON",
        "modelId": "%s",
        "objectId": "%s"
}
""" % (model_mrid, object_mrid)

In [9]:
message = """
{
        "requestType": "QUERY_OBJECT",
        "resultFormat": "JSON",
        "objectId": "_4F76A5F9-271D-9EB8-5E31-AA362D86F2C3"
}
"""

In [10]:
gapps.get_response(topic, message)

{'data': {'head': {'vars': ['property', 'value']},
  'results': {'bindings': [{'property': {'type': 'uri',
      'value': 'http://iec.ch/TC57/CIM100#Feeder.NormalEnergizingSubstation'},
     'value': {'type': 'uri',
      'value': 'http://localhost:8889/bigdata/sparql#_F1E8E479-5FA0-4BFF-8173-B375D25B0AA2'}},
    {'property': {'type': 'uri',
      'value': 'http://iec.ch/TC57/CIM100#IdentifiedObject.mRID'},
     'value': {'type': 'literal',
      'value': '_4F76A5F9-271D-9EB8-5E31-AA362D86F2C3'}},
    {'property': {'type': 'uri',
      'value': 'http://iec.ch/TC57/CIM100#IdentifiedObject.name'},
     'value': {'type': 'literal', 'value': 'ieee8500'}},
    {'property': {'type': 'uri',
      'value': 'http://iec.ch/TC57/CIM100#PowerSystemResource.Location'},
     'value': {'type': 'uri',
      'value': 'http://localhost:8889/bigdata/sparql#_9759A5A3-6813-4F79-8DA1-7E826468DF79'}},
    {'property': {'type': 'uri',
      'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'},
     'va

[Return to Top](#Table-of-Contents)

# 5. GridAPPSD-Python Shortcut Methods

A small number of simple PowerGrid Model API queries have pre-built Python functions that can be used without specifying the topic and a particular message.


## 3.1. `query_object_types`

This method is associated with the GridAPPSD connection object and returns a list of all the CIM XML classes of objects present in the Blazegraph for a particular model or all models in the database.

Allowed parameters are
* model_id (optional) - when specified, it searches only the particular model identified by the given unique mRID

__1) Query entire Blazegraph database__

Leave the arguments blank to search all models in the Blazegraph database

In [ ]:
gapps.query_object_types()

__2) Query for only a particular model__

Specify the model MRID as a python string and pass it as a parameter to the method to return only the CIM classes of objects in that particular model

In [11]:
model_mrid = "_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62" # IEEE 13 Node used for all example queries
gapps.query_object_types(model_id = model_mrid)

{'data': {'objectTypes': ['http://iec.ch/TC57/CIM100#ConnectivityNode',
   'http://iec.ch/TC57/CIM100#ACLineSegment',
   'http://iec.ch/TC57/CIM100#EnergyConsumer',
   'http://iec.ch/TC57/CIM100#TransformerTank',
   'http://iec.ch/TC57/CIM100#PowerTransformer',
   'http://iec.ch/TC57/CIM100#LoadBreakSwitch',
   'http://iec.ch/TC57/CIM100#Fuse',
   'http://iec.ch/TC57/CIM100#Breaker',
   'http://iec.ch/TC57/CIM100#PowerElectronicsConnection',
   'http://iec.ch/TC57/CIM100#EnergySource',
   'http://iec.ch/TC57/CIM100#LinearShuntCompensator',
   'http://iec.ch/TC57/CIM100#Recloser']},
 'responseComplete': True,
 'id': '408543013'}

## Query for Measurement mRIDs



In [16]:
message = {
        "requestType": "QUERY_OBJECT",
        "resultFormat": "JSON",
        "objectId": "_4F76A5F9-271D-9EB8-5E31-AA362D86F2C3"
}

gapps.get_response(topic, message)

{'data': {'head': {'vars': ['property', 'value']},
  'results': {'bindings': [{'property': {'type': 'uri',
      'value': 'http://iec.ch/TC57/CIM100#Feeder.NormalEnergizingSubstation'},
     'value': {'type': 'uri',
      'value': 'http://localhost:8889/bigdata/sparql#_F1E8E479-5FA0-4BFF-8173-B375D25B0AA2'}},
    {'property': {'type': 'uri',
      'value': 'http://iec.ch/TC57/CIM100#IdentifiedObject.mRID'},
     'value': {'type': 'literal',
      'value': '_4F76A5F9-271D-9EB8-5E31-AA362D86F2C3'}},
    {'property': {'type': 'uri',
      'value': 'http://iec.ch/TC57/CIM100#IdentifiedObject.name'},
     'value': {'type': 'literal', 'value': 'ieee8500'}},
    {'property': {'type': 'uri',
      'value': 'http://iec.ch/TC57/CIM100#PowerSystemResource.Location'},
     'value': {'type': 'uri',
      'value': 'http://localhost:8889/bigdata/sparql#_9759A5A3-6813-4F79-8DA1-7E826468DF79'}},
    {'property': {'type': 'uri',
      'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'},
     'va

In [ ]:
message = {
        "modelId": model_mrid,
        "requestType": "QUERY_OBJECT_MEASUREMENTS",
        "resultFormat": "JSON",
        "objectType": "ACLineSegment"}
obj_msr_ACline = gapps.get_response(topic, message, timeout=10)
obj_msr_ACline = obj_msr_ACline['data']
# Chose specific measurement mrid. Screen out those whose type is not PNV. For example,
obj_msr_ACline = [k for k in obj_msr_ACline if k['type'] == 'PNV']

In [ ]:
obj_msr_ACline

## 2.1. Querying for All _modelNames_ mRIDs

When passing commands through the API, it is often necessary to specify the MRID of the particular power system network model.

The PowerGrid Models API containes a function to obtain a list of all the model MRIDs stored in the Blazegrpah database. 

This method will return identical results to the JSON message explained below in section 3.

In [ ]:
gapps.query_model_names()